In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [2]:
dataset_train = pd.read_csv('MPI_HistoricalPrices.csv')
training_set = dataset_train.iloc[:, 1:2].values
dataset_train.head()

,Date,Open,High,Low,Close,Volume
0,10/08/21,3.75,3.83,3.68,3.68,19412000
1,10/07/21,3.72,3.78,3.69,3.69,14728000
2,10/06/21,3.70,3.73,3.67,3.70,32505000
3,10/05/21,3.70,3.72,3.67,3.70,9771000
4,10/04/21,3.64,3.75,3.64,3.72,7387000


In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [4]:
x_train = []
y_train = []
for i in range (60, 490):
    x_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [5]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(x_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50
14/14 [==============================] - 10s 61ms/step - loss: 0.0281
Epoch 2/50
14/14 [==============================] - 1s 61ms/step - loss: 0.0098
Epoch 3/50
14/14 [==============================] - 1s 63ms/step - loss: 0.0069
Epoch 4/50
14/14 [==============================] - 1s 65ms/step - loss: 0.0052
Epoch 5/50
14/14 [==============================] - 1s 62ms/step - loss: 0.0047
Epoch 6/50
14/14 [==============================] - 1s 62ms/step - loss: 0.0044
Epoch 7/50
14/14 [==============================] - 1s 67ms/step - loss: 0.0044
Epoch 8/50
14/14 [==============================] - 1s 67ms/step - loss: 0.0037
Epoch 9/50
14/14 [==============================] - 1s 63ms/step - loss: 0.0036
Epoch 10/50
14/14 [==============================] - 1s 68ms/step - loss: 0.0035
Epoch 11/50
14/14 [==============================] - 1s 64ms/step - loss: 0.0036
Epoch 12/50
14/14 [==============================] - 1s 63ms/step - loss: 0.0040
Epoch 13/50
14/14 [=================

In [6]:
dataset_test = pd.read_csv('Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [7]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values

inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

x_test = []

for i in range (60, 76):
    x_test.append(inputs[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test.shape[0], x_test.shape[1], 1)

predicted_stock_price = regressor.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

KeyError: 'Open'